In [25]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

# Load data from Power BI
df = pd.read_excel(r"C:\Users\Will\OneDrive\Desktop\Projects\HR analytics Power BI\HR_Data.xlsx")

# Drop non-predictive columns
df.drop(columns=['Over18'], inplace=True)

# Map binary columns
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
df['OverTime'] = df['OverTime'].map({'Yes': 1, 'No': 0})

# Clean and encode text values
df['Gender'] = df['Gender'].str.replace('"', '')  # Remove stray quotes
df['StockOptionLevel'] = df['StockOptionLevel'].replace('None', 0)
df['StockOptionLevel'] = pd.to_numeric(df['StockOptionLevel'], errors='coerce').fillna(0).astype(int)
df['AgeGroup'] = df['AgeGroup'].map({'18-25': 1, '26-35': 2, '36-45': 3, '46-55': 4, '55+': 5})
df['SalarySlab'] = df['SalarySlab'].map({'Upto 5k': 1, '5k-10k': 2, '10k-15k': 3, '15k+': 4})

# Replace 'None' and NaN in year fields
year_fields = ['TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
               'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
for col in year_fields:
    df[col] = df[col].replace('None', 0).fillna(0).astype(int)

# One-hot encode nominal columns
df = pd.get_dummies(df, columns=['BusinessTravel', 'Gender', 'MaritalStatus'], drop_first=True)

# Prepare features and target
X = df.drop(columns=['EmpID', 'Attrition'])
X = X.fillna(0)  # ⬅ Fill any remaining NaNs with 0
y = df['Attrition']

# Train logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X, y)

# Predict attrition probability
df['Attrition_Probability'] = model.predict_proba(X)[:, 1]

df_filtered= df[df['Attrition']!= 1]

# Return EmpID and probability
output = df_filtered[['EmpID', 'Attrition_Probability']]

C:\Users\Will\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
output_sorted = output.sort_values(by='Attrition_Probability',ascending=False)

In [27]:
output_sorted

,EmpID,Attrition_Probability
186,RM718,0.720416
480,RM757,0.695605
199,RM005,0.650515
469,RM1427,0.643278
74,RM842,0.626887
...,...,...
1461,RM1010,0.003094
1406,RM1077,0.003056
1043,RM717,0.002752
1398,RM895,0.002219
